In [1]:
# 노이즈 1. 문장부호 (양쪽 빈칸으로 replace 해두고, 3번으로 부호 지움)

def pad_punctuation(sentence, punc):
    for p in punc:
        sentence = sentence.replace(p, " " + p + " ")

    return sentence

sentence = "Hi, my name is john."

print(pad_punctuation(sentence, [".", "?", "!", ","]))

Hi ,  my name is john . 


In [2]:
# 노이즈 2. 대소문자 
sentence = "First, open the first chapter."

print(sentence.lower())

first, open the first chapter.


In [3]:
# 노이즈 3. 특수문자
import re

sentence = "He is a ten-year-old boy."
sentence = re.sub("([^a-zA-Z.,?!])", " ", sentence)

print(sentence)

He is a ten year old boy.


In [4]:
# From The Project Gutenberg
# (https://www.gutenberg.org/files/2397/2397-h/2397-h.htm)

corpus = \
"""
In the days that followed I learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk. 
But my teacher had been with me several weeks before I understood that everything has a name.
One day, we walked down the path to the well-house, attracted by the fragrance of the honeysuckle with which it was covered. 
Some one was drawing water and my teacher placed my hand under the spout. 
As the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly. 
I stood still, my whole attention fixed upon the motions of her fingers. 
Suddenly I felt a misty consciousness as of something forgotten—a thrill of returning thought; and somehow the mystery of language was revealed to me. 
I knew then that "w-a-t-e-r" meant the wonderful cool something that was flowing over my hand. 
That living word awakened my soul, gave it light, hope, joy, set it free! 
There were barriers still, it is true, but barriers that could in time be swept away.
""" 

def cleaning_text(text, punc, regex):
    # 노이즈 유형 (1) 문장부호 공백추가
    for p in punc:
        text = text.replace(p, " " + p + " ")

    # 노이즈 유형 (2), (3) 소문자화 및 특수문자 제거
    text = re.sub(regex, " ", text).lower()

    return text

print(cleaning_text(corpus, [".", ",", "!", "?"], "([^a-zA-Z0-9.,?!\n])"))


in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it

# 토크나이저, 형태소기반분석기 성능비교

In [5]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt
print("슝")

슝


In [6]:
tokenizer_list = [Hannanum(),Kkma(),Komoran(),Mecab(),Okt()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))

[Hannanum] 
[('코로나바이러스', 'N'), ('는', 'J'), ('2019년', 'N'), ('12월', 'N'), ('중국', 'N'), ('우한', 'N'), ('에서', 'J'), ('처음', 'M'), ('발생', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('뒤', 'N'), ('전', 'N'), ('세계', 'N'), ('로', 'J'), ('확산', 'N'), ('되', 'X'), ('ㄴ', 'E'), (',', 'S'), ('새롭', 'P'), ('은', 'E'), ('유형', 'N'), ('의', 'J'), ('호흡기', 'N'), ('감염', 'N'), ('질환', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]
[Kkma] 
[('코로나', 'NNG'), ('바', 'NNG'), ('이러', 'MAG'), ('슬', 'VV'), ('는', 'ETD'), ('2019', 'NR'), ('년', 'NNM'), ('12', 'NR'), ('월', 'NNM'), ('중국', 'NNG'), ('우', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('에', 'VV'), ('서', 'ECD'), ('처음', 'NNG'), ('발생', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('뒤', 'NNG'), ('전', 'NNG'), ('세계', 'NNG'), ('로', 'JKM'), ('확산', 'NNG'), ('되', 'XSV'), ('ㄴ', 'ETD'), (',', 'SP'), ('새', 'NNG'), ('롭', 'XSA'), ('ㄴ', 'ETD'), ('유형', 'NNG'), ('의', 'JKG'), ('호흡기', 'NNG'), ('감염', 'NNG'), ('질환', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EFN'), ('.', 'SF')]
[Komoran] 
[('코로나바이러스', 'NNP'), ('는', 'JX'), ('2019', 'SN'

# Wordpiece Model (WPM)

In [9]:
# Byte Pair Encoding(BPE)
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    
    vocab(딕셔너리) = {단어 : 횟수} 를 인풋으로 넣고
    """
    pairs = collections.defaultdict(int) # pairs 라는 빈 리스트 만들어두고
    
    for word, freq in vocab.items(): # vocab 딕셔너리를 불러서 key값인 word만 공백단위로 split 하여 symbol이라고 저장
        symbols = word.split()

        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
        
    return pairs

In [10]:
pairs = get_stats(vocab)
pairs

defaultdict(int,
            {('l', 'o'): 7,
             ('o', 'w'): 7,
             ('w', 'e'): 8,
             ('e', 'r'): 2,
             ('n', 'e'): 6,
             ('e', 'w'): 6,
             ('e', 's'): 9,
             ('s', 't'): 9,
             ('w', 'i'): 3,
             ('i', 'd'): 3,
             ('d', 'e'): 3})

In [11]:
def merge_vocab(pair, v_in):
    """
    문자 쌍(pair)과 단어 리스트(v_in)를 입력받아
    각각의 단어에서 등장하는 문자 쌍을 치환합니다.
    (하나의 글자처럼 취급)
    """
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

In [17]:
pairs = get_stats(vocab)
best = max(pairs, key=pairs.get)
print(pairs)
print('-'*20)
print(best)

defaultdict(<class 'int'>, {('low', 'e'): 2, ('e', 'r'): 2, ('ne', 'w'): 6, ('w', 'est'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est'): 3})
--------------------
('ne', 'w')


In [13]:
num_merges = 5
token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merged Vocab:", token_vocab)

>> Step 1
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w es t ': 6, 'w i d es t ': 3} 

>> Step 2
다음 문자 쌍을 치환: est
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 3
다음 문자 쌍을 치환: lo
변환된 Vocab:
 {'lo w ': 5, 'lo w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 4
다음 문자 쌍을 치환: low
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 5
다음 문자 쌍을 치환: ne
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'ne w est ': 6, 'w i d est ': 3} 

Merged Vocab: ['es', 'est', 'lo', 'low', 'ne']


In [18]:
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    """
    pairs = collections.defaultdict(int)
    
    for word, freq in vocab.items():
        symbols = word.split()

        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
        
    return pairs

def merge_vocab(pair, v_in):
    """
    문자 쌍(pair)과 단어 리스트(v_in)를 입력받아
    각각의 단어에서 등장하는 문자 쌍을 치환합니다.
    (하나의 글자처럼 취급)
    """
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merged Vocab:", token_vocab)

>> Step 1
다음 문자 쌍을 치환: es
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w es t ': 6, 'w i d es t ': 3} 

>> Step 2
다음 문자 쌍을 치환: est
변환된 Vocab:
 {'l o w ': 5, 'l o w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 3
다음 문자 쌍을 치환: lo
변환된 Vocab:
 {'lo w ': 5, 'lo w e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 4
다음 문자 쌍을 치환: low
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'n e w est ': 6, 'w i d est ': 3} 

>> Step 5
다음 문자 쌍을 치환: ne
변환된 Vocab:
 {'low ': 5, 'low e r ': 2, 'ne w est ': 6, 'w i d est ': 3} 

Merged Vocab: ['es', 'est', 'lo', 'low', 'ne']


만일 lowest라는 처음 보는 단어가 등장하더라도, 위 알고리즘을 따르면 어느 정도 의미가 파악된 low와 est의 결합으로 표현할 수 있습니다. 또 BPE의 놀라운 점은 아무리 큰 데이터도 원하는 크기로 OOV 문제없이 사전을 정의할 수 있다는 것입니다. 극단적으로 생각했을 때 알파벳 26개와 특수문자, 문장부호를 아무리 추가해도 100개 이내로 사전을 정의할 수 있죠.(물론 그러면 안 됩니다!!)

Embedding 레이어는 단어의 개수 x Embedding 차원 수 의 Weight를 생성하기 때문에 단어의 개수가 줄어드는 것은 곧 메모리의 절약으로 이어집니다. 많은 데이터가 곧 정확도로 이어지기 때문에 이런 기여는 굉장히 의미가 있습니다!

하지만 아직도! 완벽하다고는 할 수 없습니다. 만약 수많은 데이터를 사용해 만든 BPE 사전으로 모델을 학습시키고 문장을 생성하게 했다고 합시다. 그게 [i, am, a, b, o, y, a, n, d, you, are, a, gir, l]이라면, 어떤 기준으로 이들을 결합해서 문장을 복원하죠? 몽땅 한꺼번에 합쳤다간 끔찍한 일이 벌어질 것만 같습니다...

## Wordpiece Model(WPM)
이에 구글에서 BPE를 변형해 제안한 알고리즘이 바로 WPM입니다. WPM은 BPE에 대해 두 가지 차별성을 가집니다.

공백 복원을 위해 단어의 시작 부분에 언더바 _ 를 추가합니다.
빈도수 기반이 아닌 가능도(Likelihood)를 증가시키는 방향으로 문자 쌍을 합칩니다. (더 '그럴듯한' 토큰을 만들어냅니다.)
첫 번째 문항은 아주 쉬운 내용으로, 앞서 사용한 예문을 빌리면 [_i, _am, _a, _b, o, y, _a, n, d, _you, _are, _a, _gir, l]로 토큰화를 한다는 것입니다. 이렇게 하면 문장을 복원하는 과정이 1) 모든 토큰을 합친 후, 2) 언더바 _를 공백으로 치환으로 마무리되어 간편하죠.

두 번째 문항은 다소 난해하게 다가올 수도 있습니다. 직관적인 이해를 얻고 넘어가는 것을 목표로 하죠. 본 내용은 아래 논문 3절과 4절에 자세하게 나와 있습니다.

JAPANESE AND KOREAN VOICE SEARCH
(여기서 잠깐) 구글에서 이 기법을 한국어, 일본어 텍스트 처리를 위해 고려했다는 사실이 흥미롭지 않나요? 이 점은 2가지를 시사합니다.

조사, 어미 등의 활용이 많고 복잡한 한국어 같은 모델의 토크나이저로 WPM이 좋은 대안이 될 수 있다.

WPM은 어떤 언어든 무관하게 적용 가능한 language-neutral하고 general한 기법이다. 한국어 형태소 분석기처럼 한국어에만 적용 가능한 기법보다 훨씬 활용도가 크다.

그리고 가능도에 대한 개념이 낯설다면 아래 웹페이지를 방문해보시길 권장합니다.

확률(probability)과 가능도(likelihood) 그리고 최대우도추정(likelihood maximazation)
저자들은 Katz's Back-off Model 을 통해 토큰화된 사전으로부터 결합할 단어를 선정합니다. Katz's Back-off Model 은 LM에서 등장하지 않은 단어에 대해 확률을 할당할 수 있는 모델입니다. 흔히 사용되는 개념은 아니므로 "그런 게 있구나~" 하고 알아만 두시면 좋습니다. 수식을 아래에 첨부하지만 이해하지 않으셔도 괜찮습니다!

토큰화의 끝판왕으로 보이는 이 WPM은 아쉽게도 공개되어 있지는 않습니다. 대신에 구글의 SentencePiece 라이브러리를 통해 고성능의 BPE를 사용할 수 있습니다! SentencePiece에는 전처리 과정도 포함되어 있어서, 데이터를 따로 정제할 필요가 없어 간편하기까지 합니다.

이제 우리는 어떤 언어에도 OOV 발생 우려 없이 안정적으로 활용할 수 있는 멋진 토크나이징 기술을 확보했습니다. 이제는 컴퓨터가 단어사전을 안심하고 활용할 수 있겠군요!

## soynlp
이외에도 한국어를 위한 토크나이저로 soynlp를 활용할 수 있습니다. soynlp는 한국어 자연어 처리를 위한 라이브러리인데요. 토크나이저 외에도 단어 추출, 품사 판별, 전처리 기능도 제공합니다.

형태소 기반의 토크나이저가 미등록 단어에 취약하기 때문에 WordPiece Model을 사용하는 것처럼, 형태소 기반인 koNLPy의 단점을 해결하기 위해 soynlp를 사용할 수 있습니다.

soynlp의 토크나이저는 '학습데이터를 이용하지 않으면서 데이터에 존재하는 단어를 찾거나, 문장을 단어열로 분해, 혹은 품사 판별을 할 수 있는 비지도학습 접근법을 지향합니다' 라고 밝히고 있는데요.

문장에서 처음 단어를 받아들일 때 단어의 경계를 알아야 올바르게 토큰화를 할 수 있습니다. 이때 단어의 경계를 비지도학습을 통해 결정하겠다는 말이에요. 비지도학습을 통한 방법이기 때문에 미등록 단어도 토큰화가 가능합니다. 여기서 비지도학습을 가능케 하는 것이 통계적인 방법이라서 soynlp를 통계 기반 토크나이저로 분류하기도 합니다.

트와이스가 한 단어임을 인지하기 위해서 트, 트와, 트와이, 트와이스 각각 다음 글자의 확률을 계산해서 비교한다고 생각하면 좋습니다. 수학적으로 자세한 내용은 여기서는 다루지 않습니다. 다만 koNLPy외에도 soynlp가 있다는 점을 기억해 주세요.

그럼에도...
이렇게 토큰화 방법을 알아 보았지만 이걸 가지고 단어의 분산 표현을 얻는 법을 제대로 다루지는 않았습니다. 이쯤에서 이런 고민이 생기게 됩니다. 한국어라면 자동차를 _자동 / 차 로 분리되는데... 속성이 아무리 추상적이래도 보기에 차가 마시는 차인지, 달리는 차인지 도통 알 수가 없죠? 게다가 설령 토큰화가 완벽하다고 해도, 남자가 [-1, 0]인지 [1, 0]인지는 컴퓨터 입장에서는 알 도리가 없습니다.

Embedding 레이어는 선언 즉시 랜덤한 실수로 Weight 값을 채우고, 학습을 진행하며 적당히 튜닝해가는 방식으로 속성을 맞춰가지만 이는 뭔가 찜찜합니다. 토큰들이 멋지게 의미를 갖게 하는 방법은 없을까요?

# 1-6. 토큰에게 의미를 부여하기
각 토큰들이 랜덤하게 부여된 실수로 살아가지 않게, 그들끼리 유사도 연산을 할 수 있게 의미를 부여하는 알고리즘, 당연히 있습니다! 심지어 토큰화 기법보다 더 많이요! 대표적인 3가지만 알아보도록 하겠습니다. 이번 코스에선 간단하게 아이디어만 이해해도 충분합니다.

## Word2Vec
Word2Vec은 "단어를 벡터로 만든다"는 멋진 이름을 가지고 있습니다. 난 오늘 술을 한 잔 마셨어 라는 문장의 각 단어 즉, 동시에 등장하는 단어끼리는 연관성이 있다는 아이디어로 시작된 알고리즘입니다. 예문의 경우 다른 단어는 몰라도 술과 마셨어는 괜찮은 연관성을 캐치해낼 수 있겠네요.

Word2Vec에 대한 좋은 자료를 읽어보시고, 다시 얘기해보도록 하죠!


Q8. Word2Vec에는 두 가지 방식이 존재합니다. 각각 무엇인가요?  
CBOW, Skip-gram  

Q9. 윗글의 저자에 따르면 Word2Vec은 딥러닝 모델이 아닙니다. 그 이유는 무엇일까요?  
은닉층이 1개인 경우 Deep Neural Network가 아닌 Shallow Neural Network라 칭한다.  

10. Q8의 두 가지 방식 중, 어느 것이 더 성능이 좋을 것 같은지 생각해봅시다. 그리고 실제로는 어떤지 인터넷에서 찾아 적어봅시다.  
(논리적으론 CBOW가 좋아 보이지만) Skip-gram이 실제 실험에선 다소 우세하다.  


## FastText

Word2Vec은 정말 좋은 방법이지만, 연산의 빈부격차가 존재했습니다. 자주 등장하지 않는 단어는 최악의 경우 단 한 번의 연산만을 거쳐 랜덤하게 초기화된 값과 크게 다르지 않은 상태로 알고리즘이 종료될 수 있습니다. FastText는 이를 해결하기 위해 BPE와 비슷한 아이디어를 적용했습니다.

자세한 내용은 아래 웹페이지에서 만나보세요! 글 말미의 Fasttext의 등장 (2016) 파트를 읽으시면 됩니다.

Q11. 기존의 단어마다 Embedding을 할당하던 방식과 FastText의 큰 차이점은 무엇인가요? 간단하게 서술해봅시다.  
FastText는 한 단어를 n-gram의 집합이라고 보고 단어를 쪼개어 각 n-gram에 할당된 Embedding의 평균값을 사용하였다.  

## ELMo - the 1st Contextualized Word Embedding

위에 소개했던 Word Embedding 알고리즘들은 (역시나) 정말 훌륭하지만, 여전히 고질적인 문제점이 있습니다. 바로 고정적이라는 겁니다! 무슨 말이냐면, 동음이의어를 처리할 수 없다는 얘기입니다.

탐스럽고 먹음직스러웠던 사과가 이렇게 썩어버리다니 너무 아쉽네요.
당신이 저지른 실수는 사과한다고 용서될 수 있는게 아닙니다.
우리는 이 두 문장에 나오는 '사과'의 의미가 다르다는 것을 알고 있습니다. 그러나 Word2Vec이든 FastText이든 간에 이 두 문장에 나오는 사과의 워드 벡터값은 동일할 수밖에 없습니다.

분명 의미가 다른 두 '사과'의 의미를 명확하게 해석하려면 우리에게 무엇이 필요할까요? 네, 서론에서 우리는 그 답을 미리 살펴보았습니다. Context-sensitive Grammar를 따르는 자연어를 이해하려면 문맥(context)의 활용이 필수적입니다.
여기서 '사과'의 context가 되는 것은 무엇일까요? 첫 문장이라면 탐스럽고 먹음직스러운 이 될 것이고 다음 문장이라면 이렇게 썩어버리다니 가 될 것입니다. 즉, 단어의 의미 벡터를 구하기 위해서는 그 단어가 사용된 주변 단어의 맥락을 넘어서 사용된 시퀀스 전체의 맥락이 함께 고려되는 Word Embedding이 필요한 것입니다. 이런 개념을 Contextualized Word Embedding이라고 합니다.

2018년 NLP계에 큰 폭풍을 몰고 왔던 ELMo라는 모델은 데이터에 단어가 등장한 순간, 그 주변 단어 정보를 사용해 Embedding을 구축하는 개념을 처음 소개하면서 자연어처리의 획기적인 발전의 계기를 마련해 준 첫 번째 Contextualized Word Embedding 모델입니다. 정말 멋진 아이디어에요! 아래 웹페이지를 방문해 편안하게 글을 읽어보시길 바랍니다.

Q12. 양방향 LSTM을 활용한 ELMo 모델에서 Contextual Word Embedding이 되는 벡터는 어떤 벡터 3가지를 합쳐서 얻어지나요?  
기존 어휘 임베딩(입력 토큰의 word vector), 순방향 LSTM의 hidden state vector, 역방향 LSTM의 hidden state vector를 concatenate한 벡터가 ELMo의 Contextual Word Embedding이 됩니다.

지금은 모델 구조나 메커니즘을 명확하게 이해하지 않아도 좋습니다. Contextualized Word Embedding 이라는 것이 있고 ELMo나 BERT 등에 많이 사용되고 있다는 점만 기억해도 충분합니다.

이번에 배운 내용은 토큰화와 분산 표현이 중심입니다. 문장이 입력되면 적절히 토큰화를 하고 토큰을 임베딩(Embedding)을 통해 분산 표현으로 만드는 것이지요. 분산 표현은 벡터이므로 이제 인공지능에 활용할 수 있습니다.

토큰화에 사용되는 방법은 언어마다 다른데요. 문장 구성 성분이 다르기 때문입니다. 조사가 있는 한국어는 형태소 기반인 koNLPy를 주로 쓰고, WordPiece Model인 SentencePiece를 쓸 수도 있어요. 물론 그 외에 다른 방법도 있습니다.

토큰화를 마친 후 임베딩을 할 때는 토큰마다 독립적으로 만들지 않고 토큰 간의 관계성을 주입합니다. 그래야 문장을 구성할 때 적절히 사용될 수 있기 때문이에요. 이렇게 토큰 간의 관계성을 고려해서 만든 것으로는 Word2Vec, FastText 등이 있어요. 거기다가 문장의 문맥까지 고려하는 ELMo까지 등장했습니다.

이제 지금까지 배운 내용이 그려지나요? 오늘 배운 것들은 앞으로 진행될 모든 것들의 기초가 되는 부분이니 꼭 숙지하시길 권장합니다.

자연어 처리 분야는 인공지능 모델도 중요하지만 토큰화와 임베딩에 관련된 내용이 매우 중요합니다. 그러니 오늘 배용을 토대로 이 분야에 재미를 느껴 멋진 NLP 연구자로 성장하시기를 바랍니다. 고생하셨습니다!!